# Assignment 3 - Starting with the Pushbot

## 3.1 Installing the PushBot software

This Assignment assumes that you already finished Assignments 1 and 2 and your computer is already setup with Python/NumPy or [Anaconda](https://www.continuum.io/downloads) and you have a working installation of [Nengo](https://github.com/nengo/nengo) and the [Nengo-GUI](https://github.com/nengo/nengo_gui). 

Now get the code for the PushBot-Nengo interface by running 
```
git clone https://github.com/fmirus/nstbot
``` 

in the desired installation folder.
Now enter the cloned repository by 

```
cd nstbot
``` 

and change to the branch ```nengo_lab_2_lasers``` by running

```
git checkout -b nengo_lab_2_lasers origin/nengo_lab_2_lasers
```

Now install the interface by running 

```
python setup.py install
```

**Note**: if you are not in an anaconda or virtual environment this might need sudo privileges.




## 3.2 Running the first example

Open the file ```nstbot/lab_examples/nengo_lab_pushbot_example.py``` and change the last number of the IP-adress in line 5

```
bot.connect(nstbot.Socket('10.162.177.xxx'))
```

to the one written on your robot's WiFi-module on its backside, in this case 108:

<img src="imgs/pushbot.png" />

Now connect your PC to the NSTrobots_24GHz network and run the first example by 

```
nengo nstbot/lab_examples/nengo_lab_pushbot_example.py 
```

With the left resp. right slider of the motors node, you can control the left resp. right wheel of the robot. The two sliders of the behave node will activate the behaviours for driving forward/backwards or turning. Positive resp. negative values of the left slider will make the robot drive forward resp. backwards. Positive resp. negative values of the right slider will make the robot turn right resp. left. 

Play around with the controls und drive the robot around. How does the value of the y_av ensemble change based on the location of the two tracked laser points in the DVS-image? What entity can be approximated by the value encoded by the y_av ensemble?

In [1]:
!nengo nstbot/lab_examples/nengo_lab_pushbot_example.py

Starting nengo server at http://localhost:8081
Neues Fenster in aktueller Browsersitzung erstellt.
No connections remaining to the nengo_gui server.
Shutting down server...


## 3.3 Task 1: Obstacle avoidance

Extend the first example and implement a network which drives the robot around and avoids hitting obstacles based on the tracked laser points in the DVS image.

### Extension 
Use the output of your previously implemented network (or manual robot control) as "training data" for "teaching" the robot the desired behaviour. Therefore, have a look at the example network ```nstbot/lab_examples/learning_pes.py``` to get an idea of how to use the PES learning rule in Nengo.

## 3.4 Task 2: Robot cat

Change to the branch nengo_lab_1_laser by running
```
git checkout -b nengo_lab_1_laser origin/nengo_lab_1_laser
```

and reinstall this version of the code by running 
```
python setup.py install
```

again. Get one of the laser-pointer-pens and extend the first example in nstbot/examples/nengo_lab_pushbot_example.py: Implement a network which keeps the blinking stimulus of the laser-pen in the center of robot's field of view and at a certain distance.

### Extension 
Use the output of your previously implemented network (or manual robot control) as "training data" for "teaching" the robot the desired behaviour. Therefore, have a look at the example network ```nstbot/lab_examples/learning_pes.py``` to get an idea of how to use the PES learning rule in Nengo.

In [ ]:
import nengo
import numpy as np
import nstbot

bot = nstbot.PushBot()
bot.connect(nstbot.Socket('10.162.177.108'))
bot.retina(True)
bot.track_frequencies([600])
#bot.show_image()

class Bot(nengo.Network):
    def __init__(self, bot):
        super(Bot, self).__init__()
        with self:
            def bot_control(t, x):
                bot.motor(x[0], x[1], msg_period=0.1)

            self.base_ctrl = nengo.Node(bot_control, size_in=2)

            def get_points(t):
                point_array = np.array([[bot.p_x[ind], bot.p_y[ind]] for ind in range(len(bot.p_x))])
                return point_array.flatten()

            self.tracked_points = nengo.Node(get_points)

class TaskDriveFrontBack(nengo.Network):
    def __init__(self, botnet, strength=0.4):
        super(TaskDriveFrontBack, self).__init__()
        with self:
            self.activation = nengo.Ensemble(n_neurons=50,  dimensions=1, neuron_type=nengo.Direct())

        nengo.Connection(self.activation, botnet.base_ctrl, function=lambda x: [x,x] ,transform=strength)

class TaskTurn(nengo.Network):
    def __init__(self, botnet, strength=1.0):
        super(TaskTurn, self).__init__()
        with self:
            self.activation = nengo.Ensemble(n_neurons=50, dimensions=1, neuron_type=nengo.Direct())

        nengo.Connection(self.activation, botnet.base_ctrl, function=lambda x: [x,-x] ,transform=strength)

class SensorTest(nengo.Network):
    def __init__(self, botnet):
        super(SensorTest, self).__init__()
        with self:
            self.stim = nengo.Ensemble(n_neurons=200, dimensions=2, radius=1.4)

        nengo.Connection(botnet.tracked_points, self.stim, transform=1.0/128.0)


class BehaviourControl(nengo.Network):
    def __init__(self, behaviours):
        super(BehaviourControl, self).__init__()
        with self:
            self.behave = nengo.Node([0]*len(behaviours))
        for i, b in enumerate(behaviours):
            nengo.Connection(self.behave[i], b.activation, synapse=None)


model = nengo.Network(seed=2)
with model:

    motors = nengo.Node([0]*2)
    botnet = Bot(bot)
    nengo.Connection(motors, botnet.base_ctrl)

    drive_fb = TaskDriveFrontBack(botnet)
    turn_lr = TaskTurn(botnet)
    sens = SensorTest(botnet)
    bc = BehaviourControl([drive_fb, turn_lr])
    
    #diff = nengo.Ensemble(n_neurons=700, dimensions=2)
    
    #nengo.Connection(sens.stim, diff)
    #nengo.Connection(diff, diff, synapse = 0.3, function = lambda x: -x)
    
    nengo.Connection(sens.stim[1], drive_fb.activation, synapse=None,
                     function=lambda x: max(2 * (x - 0.5), 0))
    nengo.Connection(sens.stim[0], turn_lr.activation, synapse=None,
                    function=lambda x: 0.5 * (x - 0.5))